In [1]:
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.python import keras
from tensorflow.python.keras import models, layers
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D, BatchNormalization
from tensorflow.python.keras.applications import VGG16
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import classification_report, accuracy_score, roc_auc_score 

from PIL import Image
import glob

/home/sara/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/sara/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
/home/sara/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/sara/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/sara/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate 

In [2]:


#Loops through ImageDataGenerator and grabs true labels and predictions
#and reports metrics with classification_report method
def predict_and_report(gen, model):
    y_true = []
    y_pred = []
    gen.reset()
    for img, label in gen:
        #get true labels for batch and store them
        y_true.extend([int(z[1]) for z in label])
        #Get predictions as probabilities
        batch_pred = model.predict_on_batch(img)
        #turn probabilities to class labels and store
        batch_pred = np.argmax(batch_pred, axis=1)
        y_pred.extend(batch_pred)
        #break loop
        if gen.batch_index == 0:
            break
            
    print('Accuracy:', accuracy_score(y_true, y_pred))
    print('Area Under the Receiver Operating Characteristic Curve:', roc_auc_score(y_true, y_pred)) #Area under the curve
    print(classification_report(y_true, y_pred))



In [3]:
train_dir = 'data/chest_xray/train'
valid_dir = 'data/chest_xray/validation'
test_dir = 'data/chest_xray/test'

In [4]:
img_width, img_height = 75, 75
batch_size = 16
num_epochs = 1
filter_size = (3, 3)
pool_size = (2, 2)
drop_out_dense = 0.5
drop_out_conv = 0.25
padding = 'same'
#learning_rate = 0.02
#decay = learning_rate / (num_epochs - 1)

In [5]:
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=True)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    color_mode='grayscale')

Found 5216 images belonging to 2 classes.


In [6]:
validation_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=True)
validation_generator = validation_datagen.flow_from_directory(
    valid_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    color_mode='grayscale')

Found 16 images belonging to 2 classes.


In [7]:
test_datagen = ImageDataGenerator(rescale=1. / 255)
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    color_mode='grayscale')


Found 624 images belonging to 2 classes.


In [8]:
model = Sequential()

model.add(Conv2D(32, kernel_size=filter_size, activation='relu', input_shape=(img_width, img_height, 1), padding=padding))
model.add(Conv2D(32, kernel_size=filter_size, activation='relu', padding=padding))
model.add(MaxPooling2D(pool_size=pool_size))
model.add(Dropout(drop_out_conv))

model.add(Conv2D(64, kernel_size=filter_size, activation='relu', padding=padding))
model.add(Conv2D(64, kernel_size=filter_size, activation='relu', padding=padding))
model.add(MaxPooling2D(pool_size=pool_size))
model.add(Dropout(drop_out_conv))

model.add(Conv2D(96, kernel_size=filter_size, activation='relu', padding=padding))
model.add(Conv2D(96, kernel_size=filter_size, activation='relu', padding=padding))
model.add(MaxPooling2D(pool_size=pool_size))
model.add(Dropout(drop_out_conv))

model.add(Conv2D(128, kernel_size=filter_size, activation='relu', padding=padding))
model.add(Conv2D(128, kernel_size=filter_size, activation='relu', padding=padding))
model.add(MaxPooling2D(pool_size=pool_size))
model.add(Dropout(drop_out_conv))

model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dropout(drop_out_dense))
model.add(Dense(128, activation='relu'))
model.add(Dropout(drop_out_dense))
model.add(layers.Dense(2, activation='softmax'))

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 75, 75, 32)        320       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 75, 75, 32)        9248      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 37, 37, 32)        0         
_________________________________________________________________
dropout (Dropout)            (None, 37, 37, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 37, 37, 64)        18496     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 37, 37, 64)        36928     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 18, 18, 64)        0         
__________

In [9]:
SGD = keras.optimizers.SGD(lr = 0.02, decay=75e-6, momentum=0.9, nesterov=True)

model.compile(loss='categorical_crossentropy',
              optimizer='sgd',
              metrics=['accuracy'])

result = model.fit_generator(
            train_generator,
            epochs=num_epochs,
            verbose = 1,
            #class_weight= {0:.40, 1:.60},
            validation_data = validation_generator)

Epoch 1/1
326/326 [==============================] - 152s 465ms/step - loss: 0.5848 - acc: 0.7402 - val_loss: 0.7849 - val_acc: 0.5000


In [10]:
score = model.evaluate_generator(test_generator, steps=50)

print ('Test Score: ', score[0])
print ('Test Accuracy: ',score[1])

Test Score:  0.6700201570987702
Test Accuracy:  0.63375


In [11]:
filename = validation_generator.filenames
truth = validation_generator.classes
label = validation_generator.class_indices
indexlabel = dict((value, key) for key, value in label.items())

In [12]:
predicts = model.predict_generator(validation_generator, steps=validation_generator.samples/validation_generator.batch_size, verbose=1)
predict_class = np.argmax(predicts, axis=1)
errors = np.where(predict_class != truth)[0]
print("No of errors = {}/{}".format(len(errors),validation_generator.samples))

1/1 [==============================] - 0s 364ms/step
No of errors = 8/16


___

# Extract traind weights 

In [13]:
weights = model.get_weights()[0]
layer_weight=[]
for weight in weights:
    layer_weight.append(weight)

In [14]:
weights

array([[[[-1.06749147e-01, -3.13662589e-02, -6.74185455e-02,
           1.36228755e-01, -3.93731752e-05,  3.54401097e-02,
          -8.12879875e-02,  1.15229294e-01, -4.82735932e-02,
           9.21515226e-02,  2.80451868e-02, -4.82886285e-02,
          -2.63379794e-02,  3.73575054e-02,  1.02099769e-01,
           9.57153216e-02,  8.89606401e-02,  1.15599222e-01,
           3.92093398e-02, -1.40177339e-01, -1.04103964e-02,
           1.97830722e-02, -1.46923894e-02,  5.57422005e-02,
          -1.08983651e-01, -4.94999141e-02,  5.37084788e-02,
           1.36709422e-01, -3.11401822e-02, -2.30876841e-02,
           7.79863074e-02,  1.26474589e-01]],

        [[ 7.09604751e-03, -9.83311608e-03, -9.00991410e-02,
           2.49814913e-02,  5.45175411e-02,  5.63514605e-02,
           1.12756543e-01,  2.85168719e-02, -6.03976846e-02,
          -1.32157266e-01, -3.88846509e-02, -2.79108286e-02,
           7.08274022e-02,  3.90639193e-02, -9.19947028e-02,
           2.31834464e-02, -4.65109106

In [15]:
weights = model.get_weights()[1]
layer_weight=[]
for weight in weights:
    layer_weight.append(weight)

weights

array([-1.72476080e-04, -1.15964736e-04, -1.08581633e-04,  1.91211715e-04,
        2.63335503e-04, -5.33589488e-03, -6.35176722e-04, -1.94012898e-03,
       -1.38054474e-03, -1.15828283e-04,  1.54814031e-03, -7.07019120e-04,
       -2.62282859e-03, -1.43138063e-03,  2.15987660e-04,  8.17678589e-03,
        4.56048688e-03,  6.95847347e-03, -2.26172124e-04, -1.60526703e-04,
       -7.08550142e-05, -1.02534097e-04,  3.68155655e-03, -1.39730968e-04,
       -3.47852358e-04, -1.45467740e-04,  3.33836128e-04,  1.17157502e-02,
       -9.74063078e-05,  6.55317679e-04, -5.54320432e-05, -8.38293287e-04],
      dtype=float32)

In [16]:
weights = model.get_weights()[1]
layer_weight=[]
for weight in weights:
    layer_weight.append(weight)

weights

array([-1.72476080e-04, -1.15964736e-04, -1.08581633e-04,  1.91211715e-04,
        2.63335503e-04, -5.33589488e-03, -6.35176722e-04, -1.94012898e-03,
       -1.38054474e-03, -1.15828283e-04,  1.54814031e-03, -7.07019120e-04,
       -2.62282859e-03, -1.43138063e-03,  2.15987660e-04,  8.17678589e-03,
        4.56048688e-03,  6.95847347e-03, -2.26172124e-04, -1.60526703e-04,
       -7.08550142e-05, -1.02534097e-04,  3.68155655e-03, -1.39730968e-04,
       -3.47852358e-04, -1.45467740e-04,  3.33836128e-04,  1.17157502e-02,
       -9.74063078e-05,  6.55317679e-04, -5.54320432e-05, -8.38293287e-04],
      dtype=float32)

In [17]:
weights = model.get_weights()[2]
layer_weight=[]
for weight in weights:
    layer_weight.append(weight)

weights

array([[[[-0.07596248, -0.05081445,  0.02125578, ...,  0.05536578,
           0.03778863,  0.06515271],
         [-0.04043635,  0.07001254,  0.10104913, ..., -0.06794292,
          -0.08072929, -0.00170976],
         [ 0.04629036, -0.09330979, -0.0764355 , ..., -0.08574327,
           0.05688405, -0.08981317],
         ...,
         [ 0.07177496,  0.07375392,  0.02492195, ..., -0.00905579,
           0.06153452, -0.00587397],
         [-0.07811382, -0.08556741,  0.03223473, ..., -0.05248335,
           0.09697125, -0.01482551],
         [-0.0679559 , -0.09033459, -0.02862172, ..., -0.07130542,
          -0.0520356 , -0.09242786]],

        [[-0.01903463, -0.02834018, -0.09886274, ...,  0.01190681,
          -0.07087903, -0.0867764 ],
         [-0.08285289, -0.02423671, -0.01454364, ...,  0.00286979,
           0.04562303,  0.09200381],
         [-0.08978979,  0.08783319, -0.00235588, ...,  0.08116662,
           0.04614244, -0.04854098],
         ...,
         [-0.09121301,  0.08139577

In [18]:
weights = model.get_weights()[3]
layer_weight=[]
for weight in weights:
    layer_weight.append(weight)

weights

array([ 1.2753536e-02,  4.1875136e-03,  4.4154595e-03,  8.5430394e-05,
        1.2334799e-04, -1.8846247e-04, -4.3172718e-04,  2.5602002e-04,
       -1.8479308e-03, -2.7959626e-03, -2.4983851e-04,  3.6317509e-04,
        6.8467937e-04, -1.4983829e-04,  6.5461230e-03, -4.7982382e-03,
        5.4817400e-03, -8.1748818e-04, -1.0313374e-04, -1.0200125e-03,
        2.5264344e-05,  1.0359013e-02, -1.5327067e-03,  2.6811153e-04,
       -4.6622803e-04,  3.9189763e-04,  4.7817492e-04, -5.5700933e-05,
        4.0204913e-04,  3.1394514e-03,  6.1373063e-04,  4.8009946e-04],
      dtype=float32)

In [19]:
weights = model.get_weights()[4]
layer_weight=[]
for weight in weights:
    layer_weight.append(weight)

weights

array([[[[ 0.05574463, -0.04626228,  0.00800687, ..., -0.00285159,
           0.054311  , -0.04887159],
         [ 0.03571783,  0.0557369 ,  0.05414326, ..., -0.01487956,
           0.04203315, -0.0605596 ],
         [-0.04291806, -0.02026228,  0.07524741, ...,  0.08316562,
           0.06936919,  0.03610145],
         ...,
         [ 0.00841627, -0.00743742, -0.02944495, ...,  0.00500735,
          -0.04801783,  0.05209438],
         [ 0.04017094,  0.03165713, -0.01704551, ...,  0.06556681,
          -0.07949502, -0.01830218],
         [ 0.06678538, -0.01432448, -0.01661104, ...,  0.03971674,
           0.05754882,  0.04247962]],

        [[-0.07731809, -0.00048685,  0.04425297, ..., -0.06510931,
          -0.02222109, -0.08131044],
         [ 0.04750334,  0.03849132,  0.05729791, ...,  0.0071593 ,
          -0.0177833 , -0.0077048 ],
         [ 0.02742966, -0.01484483, -0.0420092 , ..., -0.07901423,
           0.03658692,  0.05254174],
         ...,
         [ 0.0214465 , -0.04921659

In [20]:
weights = model.get_weights()[5]
layer_weight=[]
for weight in weights:
    layer_weight.append(weight)

weights

array([ 1.7637512e-04,  1.8337056e-04,  3.2135954e-03, -5.4417114e-04,
        1.6841501e-03,  1.5353623e-03, -2.0246353e-05,  3.9210906e-03,
        1.2379712e-03,  3.5456422e-05,  2.7008206e-04,  2.6855189e-03,
       -4.1825784e-05,  6.1745435e-04, -7.5609292e-05,  1.5509245e-03,
        4.2125257e-03, -5.0968309e-03,  1.1647487e-03, -1.9951202e-03,
        3.0486016e-03,  3.5359897e-03,  2.5686997e-03,  6.3888036e-04,
       -2.9254984e-03,  1.9866542e-03,  5.0715293e-04,  2.5477405e-03,
        2.4850999e-03, -7.2464440e-04, -8.6406711e-05,  1.9448124e-03,
        3.6019452e-03, -1.6293912e-04, -2.6070242e-04, -1.9429456e-03,
        7.4246489e-03, -2.1360606e-04,  6.1037295e-05,  3.5196941e-03,
       -3.9466616e-04,  2.0211227e-03,  9.6823499e-03,  7.3390846e-05,
       -4.8376987e-05, -3.7891450e-03,  8.2655065e-04,  3.5571023e-03,
       -7.4030290e-04,  5.1777195e-03,  4.3903906e-03,  3.4198671e-04,
       -3.5398731e-03,  2.8176210e-03,  3.6735856e-04,  7.8082259e-05,
      

In [21]:
weights = model.get_weights()[6]
layer_weight=[]
for weight in weights:
    layer_weight.append(weight)

weights

array([[[[-6.55683428e-02,  4.42486145e-02, -4.32066321e-02, ...,
          -4.90314327e-03, -5.07786907e-02, -5.40422015e-02],
         [-4.21991237e-02,  5.68219461e-02, -5.41451126e-02, ...,
           6.68520778e-02,  6.46645203e-02, -1.83148105e-02],
         [ 2.19441541e-02,  6.54083937e-02,  3.03787272e-03, ...,
          -5.55909015e-02,  2.83308811e-02,  5.08202761e-02],
         ...,
         [ 5.51180653e-02,  1.49098993e-03, -9.91630368e-03, ...,
           6.54024854e-02,  1.62727144e-02,  2.98407152e-02],
         [-3.93034890e-02, -4.16106544e-02,  6.27713203e-02, ...,
          -6.79512415e-03,  6.84809461e-02,  5.14703058e-02],
         [ 4.05069739e-02, -8.04146752e-03, -7.01392666e-02, ...,
          -6.10788241e-02,  3.95015180e-02,  5.89794032e-02]],

        [[-1.06265433e-02, -1.26315234e-02,  6.07869215e-02, ...,
          -6.91545904e-02, -4.34031636e-02,  3.01516764e-02],
         [-6.11643195e-02,  5.27235456e-02, -1.96724422e-02, ...,
          -5.35657853e